# Ollama test

In [1]:
import cv2
import ollama
from PIL import Image
import io

# ================= 配置区域 =================
MODEL_NAME = "qwen2.5vl:32b"  

# VIDEO_PATH = "./test_videos/video_105.mp4" 
VIDEO_PATH = "./output_L2CS/video_105.mp4" 
# VIDEO_PATH = "./output_gazelle/video_105.mp4" 

# 如果视频长 10 秒，SAMPLES=10 代表每秒截一张
TARGET_FRAME_COUNT = 20 
# ===========================================

def extract_frames(video_path, target_count=TARGET_FRAME_COUNT):
    """
    从视频中均匀抽取指定数量的帧，并转换为字节流
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ 无法打开视频: {video_path}")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    duration = total_frames / fps if fps > 0 else 0
    
    print(f"🎬 视频信息: 时长 {duration:.2f}s | 总帧数 {total_frames} | FPS {fps}")
    
    # 计算抽帧间隔
    step = max(1, total_frames // target_count)
    
    frame_images = [] 
    
    count = 0
    saved_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # 按照步长抽帧
        if count % step == 0 and saved_count < target_count:
            # OpenCV 默认是 BGR，需要转为 RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)
            
            # 为了加快速度和节省显存，可以将图片缩小，例如最大边长 720 或 512
            pil_img.thumbnail((960, 540)) 
            
            # 转为字节流
            img_byte_arr = io.BytesIO()
            pil_img.save(img_byte_arr, format='JPEG', quality=90)
            image_bytes = img_byte_arr.getvalue()
            
            frame_images.append(image_bytes)
            saved_count += 1
            
        count += 1

    cap.release()
    print(f"抽取 {len(frame_images)} 帧用于分析")
    return frame_images

def analyze_video(video_path):
    frames = extract_frames(video_path, target_count=TARGET_FRAME_COUNT)
    
    if not frames:
        return

    prompt = (     
            "You are an expert in micro-expression analysis and social psychology. "
            # "Your goal is to detect 'Social Masking'—where a person hides their true negative emotions behind a fake smile.\n\n"
            "Your task is to interpret a person’s body language."
            
            "You must analyze the video frames step-by-step using the following strict reasoning pipeline:\n\n"
            
            "### Step 1: Expression Recognition\n"
            "- Observe the mouth and cheeks. Fact: [The speaker's facial expression is animated and expressive.]\n"
            # "- Check the eyes. Is it a 'Duchenne Smile' (genuine joy with eye crinkles) or a 'Social Smile' (mouth only)?\n\n"
            
            "### Step 2: Gaze Dynamics Analysis (Crucial)\n"
            "- Focus strictly on the movements of the red arrow vectors (with directions) indicating BOTH the speaker(right) and listener(left)'s gaze movements. \n"
            # "- Focus strictly on the movements of the red arrow vectors (with directions) indicating the listener (on the left)'s gaze movements. \n"
            # "- Does the gaze shifts (looking up, down, or sideways...) while the expression persists?\n"
            "- Does the gaze shifts (looking up, down, or sideways...) \n"
            
            # "- Does the speaker maintain steady eye contact with the listener?\n"
            # "- Note: 'Gaze Aversion' often indicates cognitive load, lying, or discomfort.\n\n"
            
            "### Step 3: Conflict Detection (The 'Incongruence')\n"
            "- Compare Step 1 and Step 2. Is the expression consistent with the gaze behavior? or conflict?\n"
            # "- If she is smiling but looking away/down, this is a Conflict (Incongruence).\n\n"
            
            "### Step 4: Final Inference\n"
            "- Based on the conflict, determine the true mental state.\n"
            "- Choose between:\n"
            "   A. [His body language and facial expressions indicate a positive and enthusiastic emotion, likely joy.] \n"
            "   B. [His body language and facial expressions do not indicate a positive and enthusiastic emotion, likely joy.] \n\n"
            
            "Output your reasoning for each step, and then provide the final conclusion."
        )

    # prompt = (     
    #     f"You are a socially intelligent body language expert. Your task is to interpret a person’s body language. "
    #     f"You will be given a set of video frames representing a clip with one main speaker. " 
    #     f"Follow the Task Guidelines and the Response Format.\n"            
    #     f"- You are given one fact about the speaker’s body language, and two possible interpretations.\n"
    #     f"- Think out loud about which interpretation is better given what you see in the images (2-3 sentences).\n"
    #     # add gaze
    #     f"- Please carefully observe the trajectory of the speaker's gaze (indicated by the red arrows) and eye blink in the whole video and think about it\n\n"
    #     f"- Finally, give your answer according to the Response Format.\n\n"
    #     f"Thinking out loud: <your thoughts about which interpretation is better (2-3 sentences)> \n"
    #     f"Answer: <A OR B>\n\n"
    #     f"### Fact: [She is smiling slightly.] \n\n"
    #     f"### Interpretations\n"
    #     f"A. [Based on these observations, the speaker is likely not feeling joy.] \n"
    #     f"B. [Based on these observations, the speaker is likely feeling joy.] \n\n"
    #     f"Please analyze the possible problems and give suggestions for improvement and rehabilitation actions."
    # )

    # prompt = ("describe what you have seen in the images in detail.")

    print("推理中")
    
    # 3. 发送给 Ollama:'images' 参数接受一个列表，我们把所有抽出来的帧都放进去
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=[{
                'role': 'user',
                'content': prompt,
                'images': frames  # 这里传入所有帧
            }],
                options={
                    'temperature': 0.1,
                    'num_ctx': 20000
                }
        )
        
        print("\n" + "="*20 + " 识别结果 " + "="*20)
        print(response['message']['content'])
        print("="*50)
        
    except Exception as e:
        print(f"❌ 推理出错: {e}")


analyze_video(VIDEO_PATH)

🎬 视频信息: 时长 4.00s | 总帧数 92 | FPS 23
抽取 20 帧用于分析
推理中
❌ 推理出错: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download


# Gaze-Derivative Keyframe

In [ ]:
import cv2
import numpy as np
import torch
import io
import os
import ollama
from PIL import Image
from l2cs import select_device, Pipeline, render
import pathlib
import shutil

# ================= 配置 =================
# VIDEO_PATH = '../output_L2CS/video_24.mp4'
VIDEO_PATH = '../test_videos/video_24.mp4'
MODEL_NAME = "qwen2.5vl:7b"
CWD = pathlib.Path.cwd()

# 保存关键帧的文件夹路径
OUTPUT_KEYFRAMES_DIR = './key_frames_output' 

# 阈值：只有视线角度变化超过这个值，才认为发生了 Shift
SHIFT_THRESHOLD = 0.20  
# 发送给 VLM 的最大帧数 (保持少而精)
MAX_VLM_FRAMES = 20 
# =======================================

def calculate_angular_distance(pitch1, yaw1, pitch2, yaw2):
    """计算两帧之间视线向量的欧氏距离（变化量）"""
    return np.sqrt((pitch2 - pitch1)**2 + (yaw2 - yaw1)**2)

def save_key_frames(frame_data, sorted_indices, output_dir):
    """
    保存筛选出的关键帧到本地文件夹
    """
    if os.path.exists(output_dir):
        # 如果文件夹存在，递归删除文件夹及其所有内容
        shutil.rmtree(output_dir)
        print(f"🗑️ 已清空旧文件夹: {output_dir}")
    
    # 重新创建空文件夹
    os.makedirs(output_dir)
    print(f"📁 创建新文件夹: {output_dir}")
    
    print(f"💾 正在保存 {len(sorted_indices)} 张关键帧...")
    
    saved_paths = []
    for i, idx in enumerate(sorted_indices):
        # 从 frame_data 中获取 BGR 图像
        img_bgr = frame_data[idx]['image']
        score = frame_data[idx]['score']
        
        # 文件名包含序号、原始帧号和变化分数，方便人工检查
        filename = f"keyframe_{i:02d}_frame_{idx:04d}_score_{score:.3f}.jpg"
        save_path = os.path.join(output_dir, filename)
        
        cv2.imwrite(save_path, img_bgr)
        saved_paths.append(save_path)
        
    print(f"✅ 关键帧保存完毕。路径: {output_dir}")
    return saved_paths

def process_video_smartly(video_path):
    if torch.cuda.is_available():
        device_obj = torch.device("cuda:0")
    else:
        device_obj = torch.device("cpu")
        
    # 初始化 L2CS
    gaze_pipeline = Pipeline(
        weights=CWD / 'models' / 'L2CSNet_gaze360.pkl',
        arch='ResNet50',
        device=device_obj
    )

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ 无法打开视频: {video_path}")
        return [], []

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_data = [] # 存储: {index, image, pitch, yaw, score}
    
    prev_pitch = None
    prev_yaw = None
    
    print("分析视频 Gaze 变化")
    
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret: break
        
        # 1. L2CS 推理
        with torch.no_grad():
            results = gaze_pipeline.step(frame)
        
        # 2. 绘制箭头 (保留视觉提示)
        vis_frame = render(frame, results)
        
        # 3. 计算变化量
        current_score = 0.0
        current_pitch = 0.0
        current_yaw = 0.0
        
        if len(results.pitch) > 0:
            target_idx = 0 
            current_pitch = results.pitch[target_idx].item()
            current_yaw = results.yaw[target_idx].item()
            
            if prev_pitch is not None:
                current_score = calculate_angular_distance(prev_pitch, prev_yaw, current_pitch, current_yaw)
            
            prev_pitch = current_pitch
            prev_yaw = current_yaw

        # 4. 视觉增强：写入时间戳
        time_str = f"T={frame_idx/fps:.1f}s | Frame={frame_idx}"
        cv2.putText(vis_frame, time_str, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        
        # 存储数据
        frame_data.append({
            'index': frame_idx,
            'image': vis_frame, # 已经是 BGR
            'score': current_score,
            'pitch': current_pitch,
            'yaw': current_yaw
        })
        
        frame_idx += 1
    
    cap.release()
    
    if not frame_data:
        print("❌ 未处理任何帧")
        return [], []

    # === 关键步骤：关键帧筛选策略 ===
    # 策略：首帧 + 尾帧 + 变化率最高的 Top-N 帧
    
    # 1. 找到所有变化率的峰值
    scores = [f['score'] for f in frame_data]
    # 获取变化最大的索引
    top_indices = np.argsort(scores)[-MAX_VLM_FRAMES:] 
    
    # 2. 总是包含开始和结束，保证上下文
    final_indices = set([0, len(frame_data)-1])
    
    # 3. 添加变化剧烈的帧
    for idx in top_indices:
        if scores[idx] > SHIFT_THRESHOLD: # 只有变化足够大才算
            final_indices.add(idx)
            # 同时也加入突变前一帧，方便 VLM 对比 "Before & After"
            if idx > 0: final_indices.add(idx - 1)
            
    # 4. 排序
    sorted_indices = sorted(list(final_indices))
    
    # --- 新增：保存关键帧到本地 ---
    save_key_frames(frame_data, sorted_indices, OUTPUT_KEYFRAMES_DIR)
    # ---------------------------

    selected_frames_bytes = []
    print(f"选出 {len(sorted_indices)} 个关键帧 (包含 Gaze 突变点): {sorted_indices}")
    
    for idx in sorted_indices:
        img_bgr = frame_data[idx]['image']
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(img_rgb)
        pil_img.thumbnail((960, 540)) # 缩放
        
        byte_arr = io.BytesIO()
        pil_img.save(byte_arr, format='JPEG')
        selected_frames_bytes.append(byte_arr.getvalue())
        
    return selected_frames_bytes, sorted_indices

def vlm_reasoning(frames_bytes, indices):
    if not frames_bytes:
        print("未提取到帧")
        return

    prompt = (
        f"You are an expert in micro-expression analysis. \n"
        # "You are analyzing a sequence of video frames marked with RED ARROWS indicating the gaze direction of the woman.\n\n"

        "Your task is to interpret a person’s body language."
        "You will be given a set of video frames representing a clip with one main speaker."
        "Follow the Task Guidelines."
        "Thinking out loud: <your thoughts about which interpretation is better>"

        "### STRICT ANALYSIS PIPELINE\n"
        "Follow these steps to decode the mental state:\n\n"

        "### Step 0: Fact\n"
        "- Given the fact: [She is smiling slightly.]\n"

        "1. **Vector Trajectory Tracing (The 'What')**:\n"
        "   - Compare the red arrow direction changes across timestamps.\n"
        "   - Is the arrow pointing at a fixed target ? OR is it pointing at different locations in each frame (e.g., Down -> Left -> Down-Right -> Straight)?\n"
        "   - **Key Question**: Define the 'Gaze Pattern'. Is it 'Stable/Focused' or 'Wandering/Unfocused'?\n\n"
        
        "2. **Incongruence Check (The 'Why')**:\n"
        "   - Observe her [facial expression] (Mouth/Cheeks). \n"
        "   - Contrast this with the 'Gaze Pattern' identified in Step 1. \n"
        "Is the expression consistent with the gaze behavior? or conflict?\n\n"
        
        "3. **Psychological Inference (The 'Meaning')**:\n"
        "   - Based on the specific [facial expression] + [Gaze Pattern] , what is her emotion?\n\n"
        
        "### Step 4: Final Inference\n"
        "- Based on the steps above, determine the mental state.\n"
        "- Choose between:\n"
        "   A. [Based on these observations, the speaker is likely not feeling joy.] \n"
        "   B. [Based on these observations, the speaker is likely feeling joy.] \n\n"
        
        "Output your reasoning for each step, and then provide the final conclusion."
    )

    print("🤖 发送给 VLM 推理中...")
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=[{
                'role': 'user',
                'content': prompt,
                'images': frames_bytes
            }],
            options={
                'temperature': 0.1,
                'num_ctx': 30000
            }
        )
        print("\n" + "="*30)
        print(response['message']['content'])
        print("="*30)
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    # 1. 智能提取关键帧 (Gaze 突变点) 并保存可视化结果
    frames, idxs = process_video_smartly(VIDEO_PATH)
    
    # 2. VLM 推理
    vlm_reasoning(frames, idxs)

分析视频 Gaze 变化
🗑️ 已清空旧文件夹: ./key_frames_output
📁 创建新文件夹: ./key_frames_output
💾 正在保存 33 张关键帧...
✅ 关键帧保存完毕。路径: ./key_frames_output
选出 33 个关键帧 (包含 Gaze 突变点): [0, np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(12), np.int64(13), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(48), np.int64(49), np.int64(52), np.int64(53), np.int64(56), np.int64(57), np.int64(58), 80]
🤖 发送给 VLM 推理中...

### Step 0: Fact
- Given the fact: [She is smiling slightly.]

### Step 1: Vector Trajectory Tracing (The 'What')
- **Key Question**: Define the 'Gaze Pattern'.
  - The red arrow in the frames shows a slight downward movement, then a slight rightward movement, and finally a slight upward movement.
  - The gaze appears to be wandering slightly, but it is not pointing at a fixed target. It s